# 基础调用示例

## OpenAI调用示例

In [ ]:
from openai import OpenAI

# 填入你的实际 API Key
client = OpenAI(api_key="<OpenAI API Key>", base_url="https://api.openai.com/v1")

prompt = "You are a helpful assistant"
input_text = "What is the capital of France?"
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
    {"role": "system","content": prompt},
    {
        "role": "user",
        "content": input_text
    }
    ],
    temperature=1,
    max_tokens=2048,  # 修改为 max_tokens
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# 打印响应内容
print(completion.choices[0].message.content)


The capital of France is Paris.


## DeepSeek调用示例

In [ ]:
from openai import OpenAI

# 填入你的实际 API Key
client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")

prompt = "You are a helpful assistant"
input_text = "What is the capital of France?"
completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
    {"role": "system","content": prompt},
    {
        "role": "user",
        "content": input_text
    }
    ],
    temperature=1,
    max_tokens=2048,  # 修改为 max_tokens
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# 打印响应内容
print(completion.choices[0].message.content)


The capital of France is **Paris**. Known for its rich history, iconic landmarks like the Eiffel Tower, and vibrant culture, Paris is one of the most famous cities in the world.


# 多线程调用示例
## 默认调用4o mini，可根据需要修改

In [2]:
import pandas as pd
import os
import json
import time
import logging
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from openai import OpenAI

# -------------------------------
# JSON解析模块（独立模块）
# -------------------------------
def default_json_parser(content, idx=None):
    """
    默认的 JSON 解析器：
    清理输入内容后尝试解析 JSON，
    若成功则返回完整的字典，若失败返回空字典。
    """
    try:
        # 去除代码块标记，清理内容
        cleaned_content = content.replace('```json\n', '').replace('```', '').strip()
        parsed_result = json.loads(cleaned_content)
        return parsed_result
    except json.JSONDecodeError:
        if idx is not None:
            logging.warning(f"警告: 第 {idx} 行解析 JSON 失败")
        return {}
    except Exception as e:
        if idx is not None:
            logging.error(f"错误: 第 {idx} 行解析失败 - {str(e)}")
        return {}

# -------------------------------
# 限流处理器（控制请求频率）
# -------------------------------
class RateLimitedProcessor:
    def __init__(self):
        self.request_timestamps = []
        self.MAX_RPM = 500
        self.window_size = 60  # 60秒窗口

    def _clean_old_records(self, current_time):
        cutoff_time = current_time - timedelta(seconds=self.window_size)
        self.request_timestamps = [ts for ts in self.request_timestamps if ts > cutoff_time]

    def can_make_request(self):
        """检查是否可以发起新请求"""
        current_time = datetime.now()
        self._clean_old_records(current_time)
        if len(self.request_timestamps) >= self.MAX_RPM:
            return False
        self.request_timestamps.append(current_time)
        return True

# -------------------------------
# OpenAI文本处理器
# -------------------------------
class OpenAITextProcessor:
    def __init__(self, api_key=None, model=None, base_url=None, json_parser=None):
        self.client = OpenAI(api_key=api_key,base_url=base_url)
        self.model = model
        self.rate_limiter = RateLimitedProcessor()
        self.n_workers = 14  # 优化后的线程数
        # 如果未提供自定义解析器，则使用默认解析器
        self.json_parser = json_parser if json_parser is not None else default_json_parser

    def process_batch(self, df, text_column, prompt, batch_size=20, delay=1, json_parser=None):
        """
        批量处理文本，支持灵活的 JSON 解析。
        
        参数:
            df: 包含文本数据的 DataFrame
            text_column: 文本所在的列名
            prompt: 系统提示，用于 API 调用
            batch_size: 每个批次处理的文本条数
            delay: 每次请求后的延迟（秒）
            json_parser: 可选的自定义 JSON 解析器，若不传入则使用实例内的解析器
        
        返回:
            新的 DataFrame，包含原始数据及 API 返回结果（通过 JSON 解析获得的各字段）
        """
        parser = json_parser if json_parser is not None else self.json_parser
        results = []  # 保存每次请求解析后的结果（字典形式）

        def process_chunk(chunk_data):
            chunk_results = []
            for idx, text in chunk_data:
                # 限流检测：等待直到可以发送请求
                while not self.rate_limiter.can_make_request():
                    time.sleep(0.1)
                try:
                    response = self.client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {"role": "system", "content": prompt},
                            {"role": "user", "content": text}
                        ],
                        temperature=0,
                        max_tokens=40
                    )
                    # 使用解析器处理响应内容，得到字典格式结果
                    parsed_result = parser(response.choices[0].message.content, idx)
                    chunk_results.append(parsed_result)
                    time.sleep(delay)
                except Exception as e:
                    logging.error(f"错误: 处理第 {idx} 行时发生异常: {str(e)}")
                    chunk_results.append({})
            return chunk_results

        # 将数据分成批次，保留行号信息
        chunks = [
            list(enumerate(df[text_column][i:i+batch_size]))
            for i in range(0, len(df), batch_size)
        ]

        with ThreadPoolExecutor(max_workers=self.n_workers) as executor:
            futures = list(tqdm(
                executor.map(process_chunk, chunks),
                total=len(chunks),
                desc="Processing batches"
            ))
            for chunk_results in futures:
                results.extend(chunk_results)

        # 将解析结果列表转为 DataFrame，并与原 DataFrame 合并
        df_result = df.copy().reset_index(drop=True)
        results_df = pd.json_normalize(results)
        df_result = pd.concat([df_result, results_df], axis=1)

        # 统计处理情况
        success_count = sum(1 for r in results if r)
        total_count = len(results)
        success_rate = (success_count / total_count) * 100 if total_count > 0 else 0
        logging.info(f"处理完成: 总数 {total_count}, 成功 {success_count}, 成功率 {success_rate:.2f}%")
        
        return df_result


In [3]:
prompt= "作为中文命名实体识别助手，你的任务是从政府采购公告中识别出对应的城市名称和采购金额，并进行标准化处理。\n\n- **城市名称识别**：标识并提取采购公告中出现的城市全称，如果为县级行政单位则转化为其所在的市级单位。\n- **采购金额识别与转换**：提取公告中提到的采购金额。如果金额单位为“万元”，需要将其转换为元，即将其乘以一万。\n\n# 步骤\n\n1. 阅读并解析公告内容。\n2. 查找并提取公告中的城市全称，同时识别城市行政单位。\n3. 如果是县级单位，则记录为其所在的地级市。\n4. 查找并提取采购金额，同时识别单位。\n5. 如果金额单位是“万元”，则将金额乘以一万转换为“元”。\n6. 提供一个包含城市全称和标准化后的采购金额的数据结构。\n\n# 输出格式\n\n以JSON格式提供输出，包含两个字段：\n- \"city\": \"完整的城市名称\",\n- \"amount\": 数字类型的采购金额（单位为元）\n\n# 示例\n\n- 输入：[政府采购公告文本]\n\n- 输出：\n  ```json\n  {\n    \"city\": \"北京市\",\n    \"amount\": 1000000\n  }\n  ```\n\n# 注意事项\n\n- 如果公告中未明确指出城市或金额，输出相应字段时请设为空或为0。\n- 确保金额在转换时符合单位元的精度要求。\n- 确保所有城市名都为地级市。\n-中国地市级单位包括'市''州'以及部分'县'"

In [ ]:
processor = OpenAITextProcessor(api_key="<OpenAI API Key>", base_url="https://api.openai.com/v1",model="gpt-4o-mini")
df_result = processor.process_batch(
    df=pd.read_csv("data/政府采购公告.csv"),
    text_column="公告内容",
    prompt=prompt,
    batch_size=5,
)
df_result.to_csv("data/政府采购公告_结果.csv", index=False)

Processing batches: 100%|██████████| 20/20 [00:24<00:00,  1.24s/it]
